In [1]:
import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
import json
#from modules.CARLA import carla

seed = 44

tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
# from carla.data.catalog import CsvCatalog
# import json

# with open('../data/adult_constraints.json', 'r') as f:
#     constraints = json.load(f)

# continuous = constraints['continuous']
# categorical = constraints['categorical']
# immutable = constraints['immutable']
# columns_order = constraints['features_order']

# dataset = CsvCatalog(file_path="../data/adult_prep.csv",
#                     continuous=continuous,
#                     categorical=categorical,
#                     immutables=immutable,
#                     target='income')
# dataset.df.to_csv("../data/adult_prep_test.csv",index=False)



# X_train = dataset.df_train
# Y_train = dataset.df_train
# X_test = dataset.df_test
# Y_test = dataset.df_test
# X_train

In [4]:
import json
import sklearn
from sklearn.model_selection import train_test_split

with open('../data/adult_constraints.json', 'r') as f:
    constr = json.load(f)

dataset = pd.read_csv("../data/adult_cleaned.csv")

X_train, X_test, Y_train, Y_test = train_test_split(dataset[constr['features_order_after_split']], dataset['income'], test_size=0.2, random_state=seed)
X_train.columns

Index(['capital.gain', 'education.num', 'capital.loss', 'hours.per.week',
       'age', 'workclass_?', 'workclass_Federal-gov', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Without-pay', 'marital.status_Divorced',
       'marital.status_Married-AF-spouse', 'marital.status_Married-civ-spouse',
       'marital.status_Married-spouse-absent', 'marital.status_Never-married',
       'marital.status_Separated', 'marital.status_Widowed', 'occupation_?',
       'occupation_Adm-clerical', 'occupation_Armed-Forces',
       'occupation_Craft-repair', 'occupation_Exec-managerial',
       'occupation_Farming-fishing', 'occupation_Handlers-cleaners',
       'occupation_Machine-op-inspct', 'occupation_Other-service',
       'occupation_Priv-house-serv', 'occupation_Prof-specialty',
       'occupation_Protective-serv', 'occupation_Sales',
       'occupation_Tech-suppor

In [5]:
Y_train

20789     >50K
7665     <=50K
7003     <=50K
3368     <=50K
15072    <=50K
         ...  
16955    <=50K
25773     >50K
27377    <=50K
3491     <=50K
14100    <=50K
Name: income, Length: 26048, dtype: object

In [6]:

X_train = X_train.to_numpy()
Y_train = Y_train
Y_train = pd.get_dummies(Y_train).to_numpy()
Y_train

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [7]:
X_test = X_test.to_numpy()
Y_test = Y_test
Y_test = pd.get_dummies(Y_test).to_numpy()
Y_test

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [8]:
X_train.shape

(26048, 85)

In [9]:
Y_train.shape

(26048, 2)

In [10]:
# sklearn minmaxscaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [11]:
constr['features_count_split_without_target']

85

In [12]:
from tensorflow import keras


model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((constr['features_count_split_without_target'],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2))
model.add(tf.keras.layers.Softmax())


model.compile(
    optimizer='rmsprop',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [13]:
model.fit(
    X_train, 
    Y_train,
    epochs=50,
    batch_size=256,
    validation_data=(X_test, Y_test),
    shuffle=True,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10)
    ]
)

Epoch 1/50
102/102 [==============================] - 1s 5ms/step - loss: 0.4861 - accuracy: 0.7621 - val_loss: 0.3893 - val_accuracy: 0.8251
Epoch 2/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3963 - accuracy: 0.8185 - val_loss: 0.3726 - val_accuracy: 0.8316
Epoch 3/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3780 - accuracy: 0.8262 - val_loss: 0.3616 - val_accuracy: 0.8303
Epoch 4/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3686 - accuracy: 0.8290 - val_loss: 0.3563 - val_accuracy: 0.8340
Epoch 5/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3596 - accuracy: 0.8359 - val_loss: 0.3479 - val_accuracy: 0.8385
Epoch 6/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3510 - accuracy: 0.8394 - val_loss: 0.3451 - val_accuracy: 0.8400
Epoch 7/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3454 - accuracy: 0.8413 - val_loss: 0.3413 - val_accuracy: 0.8417
Epoch 

In [14]:
model.save('../models/adult_NN.h5', overwrite=True, save_format='h5')

In [15]:
model.save('../models/adult_NN', overwrite=True)
model_test = tf.keras.models.load_model('../models/adult_NN.h5')
model_test.predict(X_test[0:2], verbose=1)

INFO:tensorflow:Assets written to: ../models/adult_NN\assets


INFO:tensorflow:Assets written to: ../models/adult_NN\assets


1/1 [==============================] - 0s 87ms/step


array([[9.321678e-01, 6.783224e-02],
       [9.990681e-01, 9.319379e-04]], dtype=float32)

In [16]:
model.load_weights('../models/adult_NN.h5')

In [17]:
X_test[0].shape

(85,)

In [18]:
model.predict(X_test[0:2], verbose=1)


1/1 [==============================] - 0s 61ms/step


array([[9.321678e-01, 6.783224e-02],
       [9.990681e-01, 9.319379e-04]], dtype=float32)

RandomForest

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import dump, load

In [20]:
classifier = RandomForestClassifier(
    random_state=seed, 
    criterion='gini', 
    n_estimators=100, 
    max_depth=15, 
    min_samples_split=3, 
    min_samples_leaf=2, 
    min_weight_fraction_leaf=0.0, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    )
classifier.fit(X_train, Y_train)

RandomForestClassifier(max_depth=15, min_samples_leaf=2, min_samples_split=3,
                       random_state=44)

In [21]:
classifier.score(X_test, Y_test)

0.8596652848149854

In [22]:
dump(classifier, '../models/adult_RF.joblib')

['../models/adult_RF.joblib']

In [23]:
clf = load('../models/adult_RF.joblib')

In [24]:
print(Y_test[0:5])
clf.predict(X_test[0:5])

[[1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]]


array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1]], dtype=uint8)

In [25]:
import pickle

with open('../models/adult_RF.pkl', 'w') as f:
    pickle.dump(classifier, f)

with open('../models/adult_RF.pkl', 'r') as f:
    clf_pickl = pickle.load(f)

clf.predict(X_test[0:5])

TypeError: file must have a 'write' attribute